<h3> Let's create our own MCP server and MCP client</h3>

In [11]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown
from openai import AsyncOpenAI

import os
load_dotenv(override=True)


True

In [12]:
from accounts import Account

In [13]:
account = Account.get("Himanshu")
account

Account(name='himanshu', balance=9993.988, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 2.004 each.], portfolio_value_time_series=[('2025-08-25 21:28:42', 10020.988), ('2025-08-25 21:28:50', 10011.988)])

In [4]:
account.buy_shares("AMZN",3,"This bookstore looks promising")

'Completed. Latest details:\n{"name": "himanshu", "balance": 9993.988, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 2.004, "timestamp": "2025-08-25 21:28:41", "rationale": "This bookstore looks promising"}], "portfolio_value_time_series": [["2025-08-25 21:28:42", 10020.988]], "total_portfolio_value": 10020.988, "total_profit_loss": 20.987999999999374}'

In [14]:
account.report()

'{"name": "himanshu", "balance": 9993.988, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 2.004, "timestamp": "2025-08-25 21:28:41", "rationale": "This bookstore looks promising"}], "portfolio_value_time_series": [["2025-08-25 21:28:42", 10020.988], ["2025-08-25 21:28:50", 10011.988], ["2025-08-25 21:37:33", 10062.988]], "total_portfolio_value": 10062.988, "total_profit_loss": 62.987999999999374}'

In [15]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 2.004,
  'timestamp': '2025-08-25 21:28:41',
  'rationale': 'This bookstore looks promising'}]

<h3>Now we will wriite an MCP server and use it directly</h3>

In [16]:
params={"command":"uv","args":["run","accounts_server.py"]}
async with MCPServerStdio(params=params,client_session_timeout_seconds=30) as server:
    mcp_tools= await server.list_tools()

In [17]:
mcp_tools

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\nArgs:\n    name: The name of the account holder\n', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, annotations=None, meta=None),
 Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\nArgs:\n    name: The name of the account holder\n', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, annotations=None, meta=None),
 Tool(name='buy_shares', title=None, description="Buy shares of a stock.\

In [ ]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Himanshu and my account is under the name Himanshu. What's my balance and my holdings?"

client = AsyncOpenAI(
    api_key=os.getenv("GEMINI_ACCESS_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=client
)

In [21]:
async with MCPServerStdio(params=params,client_session_timeout_seconds=30) as mcp_server:
    agent= Agent(name="accont_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    # with trace("account_manager"):
    #     result= await Runner.run(agent,request)
    result= await Runner.run(agent,request)
    display(Markdown(result.final_output))

OK. Your cash balance is 9993.988 and you have 3 shares of AMZN.

<h3>Now we will build out own MCP Client</h3>
<p>Writing client is not common because MCP provides a wide range of client and most of the projects use it directly<p>

In [23]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools= await get_accounts_tools_openai()

print(openai_tools)


[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\nArgs:\n    name: The name of the account holder\n', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, annotations=None, meta=None), Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\nArgs:\n    name: The name of the account holder\n', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, annotations=None, meta=None), Tool(name='buy_shares', title=None, description="Buy shares of a stock.\n\

In [27]:
request = "My name is Himanshu and my account is under the name Himanshu. What's my balance?"
agent=Agent(name="account_manager",instructions=instructions,model=model, tools=openai_tools)
result = await Runner.run(agent,request)
display(Markdown(result.final_output))

OK. Your balance is 9993.988.

In [28]:
context= await read_accounts_resource("Himanshu")
print(context)

{"name": "himanshu", "balance": 9993.988, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 2.004, "timestamp": "2025-08-25 21:28:41", "rationale": "This bookstore looks promising"}], "portfolio_value_time_series": [["2025-08-25 21:28:42", 10020.988], ["2025-08-25 21:28:50", 10011.988], ["2025-08-25 21:37:33", 10062.988], ["2025-08-25 21:49:48", 10191.988], ["2025-08-25 21:53:10", 10194.988]], "total_portfolio_value": 10194.988, "total_profit_loss": 194.98799999999937}


In [29]:
from accounts import Account
Account.get("Himanshu").report()

'{"name": "himanshu", "balance": 9993.988, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 2.004, "timestamp": "2025-08-25 21:28:41", "rationale": "This bookstore looks promising"}], "portfolio_value_time_series": [["2025-08-25 21:28:42", 10020.988], ["2025-08-25 21:28:50", 10011.988], ["2025-08-25 21:37:33", 10062.988], ["2025-08-25 21:49:48", 10191.988], ["2025-08-25 21:53:10", 10194.988], ["2025-08-25 21:53:43", 10179.988]], "total_portfolio_value": 10179.988, "total_profit_loss": 179.98799999999937}'